# Main Covid-19 data
***
Objectives:
* Country Level
* Daily Frequency


In [1]:
import numpy as np
import pandas as pd

# allow web-acces for downloading: https://stackoverflow.com/a/60671292
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from sqlalchemy import create_engine
%load_ext sql

%load_ext autoreload
%autoreload 2

In [2]:
from src.data.process_data import cleanMainDataset

> Workaround for SSL issue in venv: https://stackoverflow.com/a/60671292


## 1. Gather Data
***
We can get data from John Hopkin's University's Centre for Systems Science and Engineering's [Github page](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series).

The data is updated on a daily frequency and contains data per country (often sub-country level as well), hence suiting our requirements. The data is split in 5 files: 
* 3x global: confirmed, deaths, recovered
* 2x us: confirmed, deaths)

#### 1.1 Download the files

In [3]:
# base url to download csv data from github
base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/'

# file-specific url
files = {
    'global_confirmed' : 'time_series_covid19_confirmed_global.csv',
    'global_deaths' : 'time_series_covid19_deaths_global.csv',
    'global_recovered' : 'time_series_covid19_recovered_global.csv'
}

In [4]:
# global_confirmed
global_confirmed = pd.read_csv(base_url + files['global_confirmed'])
global_confirmed.head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,29640,30175,30451,30616,30967,31238,31517,31836,32022,32324
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,2114,2192,2269,2330,2402,2466,2535,2580,2662,2752


In [5]:
# global_deaths
global_deaths = pd.read_csv(base_url + files['global_deaths'])
global_deaths.head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,639,675,683,703,721,733,746,774,807,819
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,47,49,51,53,55,58,62,65,69,72


In [6]:
# global_recovered
global_recovered = pd.read_csv(base_url + files['global_recovered'])
global_recovered.head(2)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,9869,10174,10306,10674,12604,13934,14131,15651,16041,17331
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,1217,1250,1298,1346,1384,1438,1459,1516,1559,1592


#### 1.2 temporary storage raw files

In [7]:
raw_folder = '../../data/raw/'

global_confirmed.to_csv(raw_folder + 'global_confirmed.csv', index=False)
global_deaths.to_csv(raw_folder + 'global_deaths.csv', index=False)
global_recovered.to_csv(raw_folder + 'global_recovered.csv', index=False)

## 2. Assess Data
***

In [8]:
global_confirmed.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,29640,30175,30451,30616,30967,31238,31517,31836,32022,32324
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,2114,2192,2269,2330,2402,2466,2535,2580,2662,2752
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,12248,12445,12685,12968,13273,13571,13907,14272,14657,15070
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,855
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,197,212,212,259,267,276,284,291,315,328


### issues to address
1. Rename columns (lowercase, single words)
2. Move meta-data (`Lat`/`Long`) to separate dataframe
3. Aggregate to country level (No `Province/State` needed)
4. Move dates to a single row (unpivot - tidy data requirements)
5. Change date structure to ISO (`yyyy-mm-dd`)

#### 2.1 Rename Columns

In [9]:
global_confirmed.rename(columns = {'Country/Region' : 'country',
                                   'Province/State' : 'state',
                                   'Lat' : 'lat',
                                   'Long' : 'long'}, inplace = True)

# check
global_confirmed.head(2)

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,29640,30175,30451,30616,30967,31238,31517,31836,32022,32324
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,2114,2192,2269,2330,2402,2466,2535,2580,2662,2752


#### 2.2 Move meta-data

In [10]:
country_data = global_confirmed[['country','state','lat','long']]
country_data.head()

,country,state,lat,long
0,Afghanistan,NaN,33.0000,65.0000
1,Albania,NaN,41.1533,20.1683
2,Algeria,NaN,28.0339,1.6596
3,Andorra,NaN,42.5063,1.5218
4,Angola,NaN,-11.2027,17.8739


In [11]:
# drop lat/long
global_confirmed.drop(['lat','long'], axis = 1, inplace = True)
global_confirmed.head()

,state,country,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20
0,NaN,Afghanistan,0,0,0,0,0,0,0,0,...,29640,30175,30451,30616,30967,31238,31517,31836,32022,32324
1,NaN,Albania,0,0,0,0,0,0,0,0,...,2114,2192,2269,2330,2402,2466,2535,2580,2662,2752
2,NaN,Algeria,0,0,0,0,0,0,0,0,...,12248,12445,12685,12968,13273,13571,13907,14272,14657,15070
3,NaN,Andorra,0,0,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,855
4,NaN,Angola,0,0,0,0,0,0,0,0,...,197,212,212,259,267,276,284,291,315,328


#### 2.3 Aggregate to country level

In [12]:
# group by coutry and store in new df
confirmed_country = global_confirmed.groupby('country').sum().reset_index()

# check
assert global_confirmed[global_confirmed['country'] =='Australia']['5/7/20'].sum() == confirmed_country[confirmed_country['country'] == 'Australia']['5/7/20'].iloc[0]

In [13]:
confirmed_country.head()

,country,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,...,6/24/20,6/25/20,6/26/20,6/27/20,6/28/20,6/29/20,6/30/20,7/1/20,7/2/20,7/3/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,29640,30175,30451,30616,30967,31238,31517,31836,32022,32324
1,Albania,0,0,0,0,0,0,0,0,0,...,2114,2192,2269,2330,2402,2466,2535,2580,2662,2752
2,Algeria,0,0,0,0,0,0,0,0,0,...,12248,12445,12685,12968,13273,13571,13907,14272,14657,15070
3,Andorra,0,0,0,0,0,0,0,0,0,...,855,855,855,855,855,855,855,855,855,855
4,Angola,0,0,0,0,0,0,0,0,0,...,197,212,212,259,267,276,284,291,315,328


#### 2.4 Unpivot table

In [14]:
confirmed = confirmed_country.melt(id_vars = ["country"], 
                                  var_name = "date", 
                                value_name = "confirmed")

# check
assert confirmed[(confirmed['country'] == 'Australia') & (confirmed['date'] == '5/7/20')]['confirmed'].iloc[0] == confirmed_country[confirmed_country['country'] == 'Australia']['5/7/20'].iloc[0]

# view
confirmed.head()

,country,date,confirmed
0,Afghanistan,1/22/20,0
1,Albania,1/22/20,0
2,Algeria,1/22/20,0
3,Andorra,1/22/20,0
4,Angola,1/22/20,0


#### 2.5 Change date

In [15]:
confirmed['date'] = pd.to_datetime(confirmed['date'], format = '%m/%d/%y')

In [16]:
confirmed.tail()

,country,date,confirmed
30827,West Bank and Gaza,2020-07-03,3334
30828,Western Sahara,2020-07-03,10
30829,Yemen,2020-07-03,1240
30830,Zambia,2020-07-03,1632
30831,Zimbabwe,2020-07-03,625


#### 2.6 Remove inconsistent countries
Manual check of countries revealed that some countries are not actual countries. Examples are Kosovo (doesn't exist anymore) and two cruise-ships, hit early-on in the Covid crisis.

In [51]:
confirmed = confirmed[~confirmed['country'].isin(['Diamond Princess','MS Zaandam','Kosovo'])]
assert len(confirmed[confirmed['country'].isin(['Diamond Princess','MS Zaandam','Kosovo'])]) == 0

#### 2.7 Update names (ITERATION 2)
Whilst working with the population data (see relevant notebook), we took a closer look at the exact names as these are used to merge with this dataset. We want to clean the following names (dictionary has the current name as key and the target name as value).

In [52]:
trans_stats = {
    "Cote d'Ivoire" : 'Ivory Coast',
    'Burma' : 'Myanmar',
    'Congo (Brazzaville)' : 'Congo',
    'Congo (Kinshasa)' : 'DR Congo',
    'West Bank and Gaza' : 'State of Palestine',
    'Taiwan*' : 'Taiwan',
    'Czechia' : 'Czech Republic',
    'Korea, South' : 'South Korea',
    'US' : 'United States'}

In [53]:
confirmed.replace(trans_stats, inplace=True)
assert len(confirmed[confirmed['country'].isin(list(trans_stats.keys()))]) == 0

## 3. Automate Processing & Apply it to other sets
***
The process in **section 2** has been automated in `process_data` package and is applied to the `deaths` and `recovered` datasets. 

#### 3.1 clean all three datasets

In [59]:
global_confirmed = pd.read_csv(base_url + files['global_confirmed'])

In [60]:
confirmed = cleanMainDataset(global_confirmed, 'confirmed')
confirmed.head()

,country,date,confirmed
0,Afghanistan,2020-01-22,0
1,Albania,2020-01-22,0
2,Algeria,2020-01-22,0
3,Andorra,2020-01-22,0
4,Angola,2020-01-22,0


In [63]:
death = cleanMainDataset(global_deaths, 'deaths')
death.head()

,country,date,deaths
0,Afghanistan,2020-01-22,0
1,Albania,2020-01-22,0
2,Algeria,2020-01-22,0
3,Andorra,2020-01-22,0
4,Angola,2020-01-22,0


In [64]:
recovered = cleanMainDataset(global_recovered, 'recovered')
recovered.head()

,country,date,recovered
0,Afghanistan,2020-01-22,0
1,Albania,2020-01-22,0
2,Algeria,2020-01-22,0
3,Andorra,2020-01-22,0
4,Angola,2020-01-22,0


#### 3.2 check datasets
Start, end-date and countries

In [65]:
confirmed.agg({'date' : ['min','max','count', 'nunique'],
               'country' : 'nunique'})

,date,country
count,30710,NaN
max,2020-07-05,NaN
min,2020-01-22,NaN
nunique,166,185.0


In [66]:
death.agg({'date' : ['min','max','count', 'nunique'],
               'country' : 'nunique'})

,date,country
count,30340,NaN
max,2020-07-03,NaN
min,2020-01-22,NaN
nunique,164,185.0


In [67]:
recovered.agg({'date' : ['min','max','count', 'nunique'],
               'country' : 'nunique'})

,date,country
count,30340,NaN
max,2020-07-03,NaN
min,2020-01-22,NaN
nunique,164,185.0


#### 3.3 combine to represent target table

In [68]:
stats = confirmed.merge(death, on = ['country','date']).merge(recovered, on = ['country','date'])
stats.tail()

,country,date,confirmed,deaths,recovered
30335,State of Palestine,2020-07-03,3334,11,463
30336,Western Sahara,2020-07-03,10,1,8
30337,Yemen,2020-07-03,1240,335,536
30338,Zambia,2020-07-03,1632,30,1348
30339,Zimbabwe,2020-07-03,625,7,176


In [69]:
stats.shape

(30340, 5)

In [70]:
# manual check (with external source)
stats[stats['country'] == 'Malaysia'].tail()

,country,date,confirmed,deaths,recovered
29519,Malaysia,2020-06-29,8637,121,8334
29704,Malaysia,2020-06-30,8639,121,8354
29889,Malaysia,2020-07-01,8640,121,8375
30074,Malaysia,2020-07-02,8643,121,8437
30259,Malaysia,2020-07-03,8648,121,8446


## 4. Store Data in a DB
***
Sqlite:
* 2 tables from here: stats & population

#### 4.1 Create sqlite DB

In [71]:
# database parameters
driver = 'sqlite'
filename = '../../data/processed/covid_db.sqlite'
driver+":///"+filename

'sqlite:///../../data/processed/covid_db.sqlite'

In [72]:
# make the connection
engine = create_engine(driver+":///"+filename)
connection = engine.connect()

In [73]:
%sql sqlite:///../../data/processed/covid_db.sqlite

#### 4.2 Create stats table

In [74]:
%%sql sqlite:///../../data/processed/covid_db.sqlite
-- create the table to store our stats df
DROP TABLE IF EXISTS stats;

CREATE TABLE stats (
    country varchar NOT NULL,
    date date(1) NOT NULL,
    confirmed int,
    deaths int,
    recovered int,
    PRIMARY KEY (country, date));

Done.
Done.


[]

In [75]:
# upload df to our table
stats.to_sql('stats', con = engine, if_exists = 'append', index=False, chunksize = 1000)

In [78]:
%%sql sqlite:///../../data/processed/covid_db.sqlite
SELECT *
  FROM stats
 WHERE country = 'United States'
 ORDER BY date DESC
 LIMIT 5;

Done.


country,date,confirmed,deaths,recovered
United States,2020-07-03,2794153,129434,790404
United States,2020-07-02,2742049,128803,781970
United States,2020-07-01,2687588,128105,729994
United States,2020-06-30,2636414,127432,720631
United States,2020-06-29,2590668,126711,705203


#### 4.3 Create country table

In [77]:
# do I need this!?

#### 4.4 Create a backup

In [38]:
# + store backup...